# 1. Import Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms
from google.colab import drive
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/DeepLearning/datasets/cats_dogs/train_df.csv')
test_df = pd.read_csv('/content/drive/MyDrive/DeepLearning/datasets/cats_dogs/test_df.csv')

# 2. Preprocessing Images

In [ ]:
class customcrop:
  def __init__(self,min_size=256):
    self.min_size = min_size

  def __call__(self,image):
    # image : 3*H*W
    h = image.shape[1]
    w = image.shape[2]
    if h<self.min_size and w<self.min_size:
      new_image = transforms.Resize((self.min_size,self.min_size))(image)
      return new_image
    elif h<w:
      new_image = transforms.Resize((self.min_size,w))(image)
    else:
      new_image = transforms.Resize((h,self.min_size))(image)
    new_image = transforms.CenterCrop((256,256))(new_image)
    return new_image

train_transform = transforms.Compose([
    transforms.ToTensor(),
    customcrop(256),
    transforms.RandomResizedCrop((224,224)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    customcrop(256),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# 3. Create Dataset

In [1]:
class train_dataset(Dataset):
  def __init__(self,df,transform):
    self.df = df
    self.transform = transform

  def __len__(self):
    return len(self.df)

  def __getitem__(self,index):
    img_path = self.df.loc[index,'path']
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image = self.transform(image)
    label = self.df.loc[index,'label']

    return image,label

class test_dataset(Dataset):
  def __init__(self,df,transform):
    self.df = df
    self.transform = transform

  def __len__(self):
    return len(self.df)

  def __getitem__(self,index):
    img_path = self.df.loc[index,'path']
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image = self.transform(image)
    img0 = transforms.CenterCrop((224,224))(image)
    img1 = image[:,0:224,0:224]
    img2 = image[:,32:256,0:224]
    img3 = image[:,0:224,32:256]
    img4 = image[:,32:256,32:256]
    img0_r = transforms.RandomHorizontalFlip(p=1)(img0)
    img1_r = transforms.RandomHorizontalFlip(p=1)(img1)
    img2_r = transforms.RandomHorizontalFlip(p=1)(img2)
    img3_r = transforms.RandomHorizontalFlip(p=1)(img3)
    img4_r = transforms.RandomHorizontalFlip(p=1)(img4)
    label = self.df.loc[index,'label']

    return img0,img1,img2,img3,img4,img0_r,img1_r,img2_r,img3_r,img4_r,label


NameError: name 'Dataset' is not defined

# 4. Create Model

In [ ]:
class AlexNet(nn.Module):
  def __init__(self,num_classes):
    super(AlexNet,self).__init__()
    self.convlayer1 = nn.Sequential(
        nn.Conv2d(3,96,kernel_size=11,stride=4,padding=2), # B*96*55*55
        nn.ReLU(inplace=True),
        nn.LocalResponseNorm(size=5,alpha=1e-4,beta=0.75,k=2),
        nn.MaxPool2d(kernel_size=3,stride=2) # B*96*27*27
    )
    self.convlayer2 = nn.Sequential(
        nn.Conv2d(96,256,kernel_size=5,padding=2), # B*256*27*27
        nn.ReLU(inplace=True),
        nn.LocalResponseNorm(size=5,alpha=1e-4,beta=0.75,k=2),
        nn.MaxPool2d(kernel_size=3,stride=2) # B*256*13*13
    )
    self.convlayer3 = nn.Sequential(
        nn.Conv2d(256,384,kernel_size=3,padding=1), # B*384*13*13
        nn.ReLU(inplace=True)
    )
    self.convlayer4 = nn.Sequential(
        nn.Conv2d(384,384,kernel_size=3,padding=1), # B*384*13*13
        nn.ReLU(inplace=True)
    )
    self.convlayer5 = nn.Sequential(
        nn.Conv2d(384,256,kernel_size=3,padding=1), # B*256*13*13
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3,stride=2), # B*256*6*6
        nn.AdaptiveAvgPool2d((6,6))
    )
    self.fc = nn.Sequential(
        nn.Dropout(),
        nn.Linear(256*6*6,4096),
        nn.ReLU(inplace=True),
        nn.Dropout(),
        nn.Linear(4096,4096),
        nn.ReLU(inplace=True),
        nn.Linear(4096,num_classes)
    )

  def forward(self,x):
    out = self.convlayer1(x)
    out = self.convlayer2(out)
    out = self.convlayer3(out)
    out = self.convlayer4(out)
    out = self.convlayer5(out)
    out = out.reshape(-1,256*6*6)
    out = self.fc(out)
    return out

# 5. Train

In [ ]:
epoch = 10
batch_size = 4
learning_rate = 1e-2

In [ ]:
train_data = train_dataset(train_df,train_transform)
test_data = test_dataset(test_df,test_transform)

train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True,num_workers=2,drop_last=True)
test_loader = DataLoader(test_data,batch_size=1,shuffle=False,num_workers=2,drop_last=True)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = AlexNet(num_classes=2)
model = model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=learning_rate,momentum=0.9,weight_decay=5e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode='min',factor=0.1,patience=10,verbose=True)

for i in tqdm(range(epoch)):
  train_loss_list = []
  model.train()
  for image,label in train_loader:
    optimizer.zero_grad()
    image = image.to(device)
    label = label.to(device)
    output = model(image)
    loss = loss_fn(output,label)
    loss.backward()
    optimizer.step()
    train_loss_list.append(loss.item())

  test_loss_list = []
  test_acc_list = []
  model.eval()
  for img0,img1,img2,img3,img4,img0_r,img1_r,img2_r,img3_r,img4_r,label in test_loader:
    image = torch.cat([img0,img1,img2,img3,img4,img0_r,img1_r,img2_r,img3_r,img4_r],dim=0)
    label = torch.full((10,),int(label))
    image = image.to(device)
    label = label.to(device)
    output = model(image)
    loss = loss_fn(output,label)
    test_loss_list.append(loss.item())
    pred = torch.argmax(output,dim=1)
    acc = (pred==label).sum().item()/10
    test_acc_list.append(acc)

  train_loss = np.mean(train_loss_list)
  test_loss = np.mean(test_loss_list)
  test_acc = np.mean(test_acc_list)
  scheduler.step(test_loss)

  print('{}th epoch - train loss : {:.3f} / test loss : {:.3f} / test acc : {:.3f}'.format(i,train_loss,test_loss,test_acc))


  0%|          | 0/10 [00:00<?, ?it/s]

0th epoch - train loss : 0.701 / test loss : 0.703 / test acc : 0.500
1th epoch - train loss : 0.696 / test loss : 0.693 / test acc : 0.500
2th epoch - train loss : 0.697 / test loss : 0.693 / test acc : 0.501
3th epoch - train loss : 0.697 / test loss : 0.694 / test acc : 0.500
4th epoch - train loss : 0.692 / test loss : 0.699 / test acc : 0.500
5th epoch - train loss : 0.697 / test loss : 0.693 / test acc : 0.500
6th epoch - train loss : 0.696 / test loss : 0.694 / test acc : 0.500
7th epoch - train loss : 0.699 / test loss : 0.695 / test acc : 0.500
8th epoch - train loss : 0.695 / test loss : 0.694 / test acc : 0.500
9th epoch - train loss : 0.691 / test loss : 0.697 / test acc : 0.500
